<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#num_unq-aggregations" data-toc-modified-id="num_unq-aggregations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>num_unq aggregations</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import vaex

In [2]:
df = vaex.open('./data/user_logs_split*')

In [3]:
dict_names = dict(zip(list(range(9)), ['msno','date','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']))

In [4]:
dict_names

{0: 'msno',
 1: 'date',
 2: 'num_25',
 3: 'num_50',
 4: 'num_75',
 5: 'num_985',
 6: 'num_100',
 7: 'num_unq',
 8: 'total_secs'}

In [5]:
for x in dict_names.keys():
    df.rename(str(x), dict_names[x])

In [6]:
sample = df[:50000]

In [7]:
sample['year'] = sample['date'] // 10000
sample['month'] = (sample['date'] // 100) % 100
sample['day'] = sample['date'] % 100

In [8]:
sample['year'] = sample['year'].astype(str)
sample['month'] = sample['month'].astype(str).str.zfill(2)
sample['day'] = sample['day'].astype(str).str.zfill(2)

In [9]:
sample['date_formatted'] = sample['year'] + '-' + sample['month'] + '-' + sample['day']
sample['date_formatted'] = sample['date_formatted'].astype('datetime64')

In [10]:
temp = sample.groupby(['msno', vaex.BinnerTime(sample['date_formatted'], resolution='M', every=6)]).agg(
    {'total_uid_seconds_month': vaex.agg.sum(sample.total_secs)})

In [11]:
res = temp.to_pandas_df()

In [12]:
res

,msno,date_formatted,total_uid_seconds_month
0,WW6mBsp6J76Z4P4IE/3gikg23E5xOQAnemGBIkCZyVc=,2014-12-01,14529.872
1,WW6mBsp6J76Z4P4IE/3gikg23E5xOQAnemGBIkCZyVc=,2015-06-01,14660.008
2,WW6mBsp6J76Z4P4IE/3gikg23E5xOQAnemGBIkCZyVc=,2015-12-01,22235.482
3,WW6mBsp6J76Z4P4IE/3gikg23E5xOQAnemGBIkCZyVc=,2016-06-01,1781.418
4,WW6mBsp6J76Z4P4IE/3gikg23E5xOQAnemGBIkCZyVc=,2016-12-01,9703.137
...,...,...,...
26494,P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=,2014-12-01,12752.983
26495,P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=,2015-06-01,9773.071
26496,P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=,2015-12-01,12873.443
26497,P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=,2016-06-01,16944.040


In [13]:
del temp

In [14]:
res.msno.nunique()

9460

In [10]:
import os

In [11]:
path = os.path.abspath('./data/train.csv')

In [24]:
path

'/home/achraf/kaggle_challenge/project/data/train.csv'

In [31]:
splitted_path = path.split('.')[0]

In [12]:
import pandas as pd

In [33]:
for i, chunk in enumerate(pd.read_csv(path, chunksize=5e4, header=None)):
    df = vaex.from_pandas(chunk)
    df.export_hdf5(splitted_path + '_split_' + str(i) + '.hdf5')

In [34]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00')
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00')
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00')
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00')
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00')
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00')
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00')
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00')
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00')


In [35]:
temp = sample.groupby(['msno', vaex.BinnerTime(sample['date_formatted'], resolution='M', every=6)]).agg(
    {'total_seconds_6_months': vaex.agg.sum(sample.total_secs),
    'avg_seconds_6_months': vaex.agg.mean(sample.total_secs)})

In [36]:
temp

#,msno,date_formatted,total_seconds_6_months,average_seconds_6_months
0,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2014-12,75637.76699999999,25212.588999999996
1,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2015-06,72942.37599999999,18235.593999999997
2,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2015-12,64750.97900000001,21583.65966666667
3,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2016-06,78364.671,19591.16775
4,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2016-12,13442.364000000001,6721.182000000001
...,...,...,...,...
"26,494",'O7ueKy1Wo8k64CWkxBnG4iMje0m9t3KfZMpiS2CdCSM=',2015-12,48658.625,16219.541666666666
"26,495",'O7ueKy1Wo8k64CWkxBnG4iMje0m9t3KfZMpiS2CdCSM=',2016-06,13644.572,6822.286
"26,496",'O7ueKy1Wo8k64CWkxBnG4iMje0m9t3KfZMpiS2CdCSM=',2016-12,4588.589,4588.589
"26,497",'nSTyR6e5MmKwIvWplfmBFx8izI94Xj9KMeFiCacJbVY=',2015-12,398.754,398.754


In [37]:
temp.shape

(26499, 4)

In [41]:
sample.groupby('msno').agg({'avg_num_unq': vaex.agg.mean(sample.num_unq)})

#,msno,avg_num_unq
0,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',23.785714285714285
1,'PjIozBWbtHFNte0hWowl7lkfoJyIHOkXffw2xKCXMzQ=',7.0
2,'kcCTUO3AS8suFa0ONtaJAr/QBsXxviihQfoAFCq1VbM=',22.916666666666668
3,'OZFfHmeW6lCkxI12tG8nER17EO1LbzI4iKptFjDvFI8=',24.666666666666668
4,'S1EnE+jqaXaVlPRnOyignxkKVjSoXt+R1I20BbDNQf4=',19.666666666666668
...,...,...
"9,455",'U9wZ+C6jSSjcNGwO5b5vjKlJ8Ip9lsG1NxPdGrjAUpc=',65.14285714285714
"9,456",'QawonBRmAUdH76bsSUhMv2S75ABPbhB8Wu79PIV3BrE=',17.0
"9,457",'PEX0oJ6D87UM42/BAVVhX8fWwTq/3EfDLCVJ1x3xhYA=',13.222222222222221
"9,458",'pxrhl5Ub4GcRP0w5OpPa9CnLZm7rsIcR1zN5xYNadkI=',27.833333333333332


In [38]:
sample['total_secs'].

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00')
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00')
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00')
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00')
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00')
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00')
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00')
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00')
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00')


In [39]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00')
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00')
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00')
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00')
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00')
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00')
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00')
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00')
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00')


In [45]:
sample.groupby('msno')

In [46]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00')
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00')
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00')
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00')
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00')
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00')
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00')
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00')
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00')


In [15]:
t1 = sample.groupby('msno').agg({'max_date_uid': vaex.agg.max(sample.date)})

In [16]:
sample = sample.join(t1, how="left", on="msno")

In [17]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted,max_date_uid
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00'),20150709
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00'),20150709
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00'),20170106
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00'),20170106
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00'),20170106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00'),20161123
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00'),20151212
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00'),20151212
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00'),20151212


In [88]:
(sample['max_date_uid'] // 10000) * 12

Expression = ((max_date_uid // 10000) * 12)
Length: 50,000 dtype: int64 (expression)
----------------------------------------
    0  24180
    1  24180
    2  24204
    3  24204
    4  24204
    ...     
49995  24192
49996  24180
49997  24180
49998  24180
49999  24180

In [18]:
def to_days(series):
    return series % 100 + ((series // 100) % 100) * 30 + (series // 10000) * 365

In [19]:
to_days(sample['max_date_uid'])

Expression = (((max_date_uid % 100) + (((max_date_uid // 100) % 100) *...
Length: 50,000 dtype: int64 (expression)
----------------------------------------
    0  735694
    1  735694
    2  736241
    3  736241
    4  736241
     ...     
49995  736193
49996  735847
49997  735847
49998  735847
49999  735847

In [102]:
res1 = (to_days(sample['max_date_uid']) - to_days(sample['date'])) > 6 * 30

In [95]:
sample.select(res1)

In [103]:
sample[res1]

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted,max_date_uid
0,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00'),20170106
1,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00'),20170106
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00'),20170106
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150702,4,0,1,1,33,10,10021.52,'2015','07','02',Timestamp('2015-07-02 00:00:00'),20170106
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150830,3,1,0,0,4,7,1119.555,'2015','08','30',Timestamp('2015-08-30 00:00:00'),20170106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"29,012",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20151229,1,1,1,0,9,10,2381.3360000000002,'2015','12','29',Timestamp('2015-12-29 00:00:00'),20161123
"29,013",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20160221,5,1,1,1,6,12,2260.229,'2016','02','21',Timestamp('2016-02-21 00:00:00'),20161123
"29,014",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20160414,10,2,0,1,7,17,2291.386,'2016','04','14',Timestamp('2016-04-14 00:00:00'),20161123
"29,015",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00'),20151212


In [20]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted,max_date_uid
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00'),20150709
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00'),20150709
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00'),20170106
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00'),20170106
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00'),20170106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00'),20161123
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00'),20151212
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00'),20151212
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00'),20151212


In [81]:
s1, s2 = 20150720, 20140930

In [82]:
(2015 * 365 + 7 * 12 + 20) - (2014 * 365 + 9 * 12 + 30)

331

In [83]:
330/ 30

11.0

In [61]:
res = sample.groupby(['msno',
                vaex.BinnerTime(sample.date_formatted,
                                resolution='M',
                                every=2)]).agg({'avg_num_unq_2_weeks': vaex.agg.mean(sample.num_unq)})

In [66]:
res

#,msno,date_formatted,avg_num_unq_biweekly
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',2015-04,1.0
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',2015-06,11.0
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2014-12,36.0
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2015-02,27.0
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2015-04,38.0
...,...,...,...
"47,760",'O7ueKy1Wo8k64CWkxBnG4iMje0m9t3KfZMpiS2CdCSM=',2016-10,24.0
"47,761",'O7ueKy1Wo8k64CWkxBnG4iMje0m9t3KfZMpiS2CdCSM=',2016-12,37.0
"47,762",'nSTyR6e5MmKwIvWplfmBFx8izI94Xj9KMeFiCacJbVY=',2016-04,12.0
"47,763",'nSTyR6e5MmKwIvWplfmBFx8izI94Xj9KMeFiCacJbVY=',2016-06,16.0


In [104]:
sample

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,year,month,day,date_formatted,max_date_uid
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,'2015','05','13',Timestamp('2015-05-13 00:00:00'),20150709
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,'2015','07','09',Timestamp('2015-07-09 00:00:00'),20150709
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,'2015','01','05',Timestamp('2015-01-05 00:00:00'),20170106
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,'2015','03','06',Timestamp('2015-03-06 00:00:00'),20170106
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,'2015','05','01',Timestamp('2015-05-01 00:00:00'),20170106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"49,995",'w8Wv1H79RS3LL5Q75COIm/rt2V8B6uOr/7V8Wzw3SKM=',20161123,2,1,0,1,10,14,3048.51,'2016','11','23',Timestamp('2016-11-23 00:00:00'),20161123
"49,996",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150321,2,0,0,0,10,9,2534.337,'2015','03','21',Timestamp('2015-03-21 00:00:00'),20151212
"49,997",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150601,0,1,0,0,0,1,98.31299999999999,'2015','06','01',Timestamp('2015-06-01 00:00:00'),20151212
"49,998",'LdvtNHmd5f5iJSxFroRbmaP5VkiDc7PEv6kzfOq8/j8=',20150902,0,0,0,0,14,13,3521.985,'2015','09','02',Timestamp('2015-09-02 00:00:00'),20151212


# num_unq aggregations

In [112]:
sample.groupby(['msno', vaex.BinnerTime(sample.date_formatted, resolution='W', every=2)]).agg(
    {'avg_num_unq_2_weeks': vaex.agg.mean(sample.num_unq), 'sum_num_unq_2_weeks': vaex.agg.sum(sample.num_unq)})

#,msno,date_formatted,avg_num_unq_2_weeks,sum_num_unq_2_weeks
0,'XSKxTn/s9fzMDRLHoI8Ouc8CSlW6PYT071JaBojv+p0=',2015-01-22,43.0,43
1,'XSKxTn/s9fzMDRLHoI8Ouc8CSlW6PYT071JaBojv+p0=',2015-03-19,150.0,150
2,'XSKxTn/s9fzMDRLHoI8Ouc8CSlW6PYT071JaBojv+p0=',2015-05-14,22.0,22
3,'XSKxTn/s9fzMDRLHoI8Ouc8CSlW6PYT071JaBojv+p0=',2015-08-06,30.0,30
4,'XSKxTn/s9fzMDRLHoI8Ouc8CSlW6PYT071JaBojv+p0=',2015-10-01,31.0,31
...,...,...,...,...
"49,995",'eWCMedKf+yid9kFcC4kn1cBE2xEFdwojZTC5pxnznaU=',2015-10-29,98.0,98
"49,996",'nUC1vXdNQKZap5i9aKiCGb5nV0d5JmrtNKBI1ybZNwo=',2016-11-10,1.0,1
"49,997",'w6RFLkszYXEJUNw5rkJ51/D1KbXAdkyW8oBnOVuELa4=',2016-08-18,10.0,10
"49,998",'w6RFLkszYXEJUNw5rkJ51/D1KbXAdkyW8oBnOVuELa4=',2016-11-24,5.0,5


In [113]:
sample.groupby(['msno', vaex.BinnerTime(sample.date_formatted, resolution='M', every=1)]).agg(
    {'avg_num_unq_monthly': vaex.agg.mean(sample.num_unq), 'sum_num_unq_monthly': vaex.agg.sum(sample.num_unq)})

#,msno,date_formatted,avg_num_unq_monthly,sum_num_unq_monthly
0,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2015-11,34.0,34
1,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-01,16.0,16
2,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-03,10.0,10
3,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-06,25.0,25
4,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-08,1.0,1
...,...,...,...,...
"49,995",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-03,115.0,115
"49,996",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-09,40.0,40
"49,997",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-10,149.0,149
"49,998",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-12,86.0,86


In [114]:
sample.groupby(['msno', vaex.BinnerTime(sample.date_formatted, resolution='M', every=3)]).agg(
    {'avg_num_unq_3_months': vaex.agg.mean(sample.num_unq), 'sum_num_unq_3_months': vaex.agg.sum(sample.num_unq)})

#,msno,date_formatted,avg_num_unq_3_months,sum_num_unq_3_months
0,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2015-09,34.0,34
1,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2015-12,16.0,16
2,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-03,10.0,10
3,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-06,13.0,26
4,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-09,6.0,6
...,...,...,...,...
"39,118",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2015-09,26.0,26
"39,119",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2015-12,24.0,24
"39,120",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-03,115.0,115
"39,121",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-09,94.5,189


In [115]:
sample.groupby(['msno', vaex.BinnerTime(sample.date_formatted, resolution='M', every=6)]).agg(
    {'avg_num_unq_6_months': vaex.agg.mean(sample.num_unq), 'sum_num_unq_6_months': vaex.agg.sum(sample.num_unq)})

#,msno,date_formatted,avg_num_unq_6_months,sum_num_unq_6_months
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',2014-12,1.0,1
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',2015-06,11.0,11
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2014-12,33.666666666666664,101
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2015-06,7.333333333333333,22
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',2015-12,40.666666666666664,122
...,...,...,...,...
"26,494",'eWCMedKf+yid9kFcC4kn1cBE2xEFdwojZTC5pxnznaU=',2015-06,98.0,98
"26,495",'nUC1vXdNQKZap5i9aKiCGb5nV0d5JmrtNKBI1ybZNwo=',2016-06,1.0,1
"26,496",'w6RFLkszYXEJUNw5rkJ51/D1KbXAdkyW8oBnOVuELa4=',2016-06,10.0,10
"26,497",'w6RFLkszYXEJUNw5rkJ51/D1KbXAdkyW8oBnOVuELa4=',2016-12,5.0,5


In [21]:
mappings_resolutions = {'Y': 'year', 'M': 'month', 'W': 'week'}

In [38]:
def return_agg_by_time(df, uid_col, time_col, to_agg_col, resolution, every=1):
    res_agg_name = to_agg_col + '_' + str(every) + '_' + mappings_resolutions[resolution]
    return df.groupby([uid_col, vaex.BinnerTime(df[time_col], resolution=resolution, every=every)]).agg(
        {'avg_' + res_agg_name: vaex.agg.mean(df[to_agg_col]),
         'sum_' + res_agg_name: vaex.agg.sum(df[to_agg_col]),
         'max_' + res_agg_name: vaex.agg.max(df[to_agg_col]),
         'min_' + res_agg_name: vaex.agg.min(df[to_agg_col]),
         'std_' + res_agg_name: vaex.agg.std(df[to_agg_col])
        }
    ).to_pandas_df()

In [39]:
r1 = return_agg_by_time(sample, 'msno', 'date_formatted', 'num_unq', resolution='Y', every=1)

In [24]:
r2 = sample.groupby(['msno', vaex.BinnerTime(sample.date_formatted, resolution='Y', every=1)]).agg(
    {'avg_num_unq_yearly': vaex.agg.mean(sample.num_unq), 'sum_num_unq_yearly': vaex.agg.sum(sample.num_unq),
    'min_num_unq_yearly': vaex.agg.min(sample.num_unq), 'max_num_unq_yearly': vaex.agg.max(sample['num_unq'])})

In [25]:
r1.to_pandas_df().sort_values('msno')

,msno,date_formatted,avg_num_unq_1_year,sum_num_unq_1_year,max_num_unq_1_year,min_num_unq_1_year,std_num_unq_1_year
14134,++Re/XY4v6500lA5vyrUA/WPsAfXqv1A/4d0FCbvpSI=,2017-01-01,57.000000,57,57,57,0.000000
9245,++am6f+rLDE3gjQM7pKLVAthwCgaI46WHbTNuKtgpbI=,2016-01-01,6.000000,6,6,6,0.000000
9244,++am6f+rLDE3gjQM7pKLVAthwCgaI46WHbTNuKtgpbI=,2015-01-01,94.000000,94,94,94,0.000000
1204,++wYPELMJ2byTXrC7gO4p96cSGNrLmGn8zNHXI97g6o=,2016-01-01,20.000000,140,44,4,13.918128
1203,++wYPELMJ2byTXrC7gO4p96cSGNrLmGn8zNHXI97g6o=,2015-01-01,20.857143,146,34,2,12.310606
...,...,...,...,...,...,...,...
13861,zyJc6C/9nxPksg1QqAH6x5Hg+aY7c6hHzGJm43bxI4g=,2016-01-01,1.000000,1,1,1,0.000000
10551,zyv/tgkfq6v1kmCG1zkblK91VDVwjIVo9l68ixn5d70=,2017-01-01,26.000000,26,26,26,0.000000
10550,zyv/tgkfq6v1kmCG1zkblK91VDVwjIVo9l68ixn5d70=,2016-01-01,26.666667,80,43,9,13.912425
13477,zzVLNBeDA/LeQX9kVCwgd/mr0YAbrdUY/jkoJvNoW/c=,2017-01-01,8.000000,8,8,8,0.000000


In [26]:
r2.to_pandas_df().sort_values('msno')

,msno,date_formatted,avg_num_unq_yearly,sum_num_unq_yearly,min_num_unq_yearly,max_num_unq_yearly
2572,++Re/XY4v6500lA5vyrUA/WPsAfXqv1A/4d0FCbvpSI=,2017-01-01,57.000000,57,57,57
5259,++am6f+rLDE3gjQM7pKLVAthwCgaI46WHbTNuKtgpbI=,2016-01-01,6.000000,6,6,6
5258,++am6f+rLDE3gjQM7pKLVAthwCgaI46WHbTNuKtgpbI=,2015-01-01,94.000000,94,94,94
12613,++wYPELMJ2byTXrC7gO4p96cSGNrLmGn8zNHXI97g6o=,2016-01-01,20.000000,140,4,44
12612,++wYPELMJ2byTXrC7gO4p96cSGNrLmGn8zNHXI97g6o=,2015-01-01,20.857143,146,2,34
...,...,...,...,...,...,...
595,zyJc6C/9nxPksg1QqAH6x5Hg+aY7c6hHzGJm43bxI4g=,2016-01-01,1.000000,1,1,1
2127,zyv/tgkfq6v1kmCG1zkblK91VDVwjIVo9l68ixn5d70=,2016-01-01,26.666667,80,9,43
2128,zyv/tgkfq6v1kmCG1zkblK91VDVwjIVo9l68ixn5d70=,2017-01-01,26.000000,26,26,26
1753,zzVLNBeDA/LeQX9kVCwgd/mr0YAbrdUY/jkoJvNoW/c=,2016-01-01,57.250000,229,11,151


In [27]:
yearly_aggs_num_unq = return_agg_by_time(sample, 'msno', 'date_formatted', 'num_unq', resolution='Y', every=1)

In [28]:
yearly_aggs_num_unq

#,msno,date_formatted,avg_num_unq_1_year,sum_num_unq_1_year,max_num_unq_1_year,min_num_unq_1_year,std_num_unq_1_year
0,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2015,84.42857142857143,591,130,50,27.259898033857844
1,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2016,63.5,508,102,17,27.847800631288642
2,'RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=',2017,35.0,35,35,35,0.0
3,'KsiXgnQvL6mb1DnuEF/eljW9cnyZrJTWr3Fc3VPQB9Y=',2015,2.5,5,3,2,0.5
4,'JPB+YKy0nEKCG0JCyv65x4SRjIqvH0zF4ouN7TW3jhI=',2015,2.0,2,2,2,0.0
...,...,...,...,...,...,...,...
"17,661",'uxL/GrAO/17mWRBAb2s+o23QE4w1FfOIcLvQPc16cYo=',2016,36.5,146,111,6,43.22325762827231
"17,662",'uxL/GrAO/17mWRBAb2s+o23QE4w1FfOIcLvQPc16cYo=',2017,124.0,124,124,124,0.0
"17,663",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2015,45.5,182,140,1,55.274315916164895
"17,664",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016,82.8,414,149,24,46.30075593335384


In [31]:
return_agg_by_time(sample, 'msno', 'date_formatted', 'num_unq', resolution='M', every=1)

#,msno,date_formatted,avg_num_unq_1_month,sum_num_unq_1_month,max_num_unq_1_month,min_num_unq_1_month,std_num_unq_1_month
0,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2015-11,34.0,34,34,34,0.0
1,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-01,16.0,16,16,16,0.0
2,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-03,10.0,10,10,10,0.0
3,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-06,25.0,25,25,25,0.0
4,'kj6EKhEip7IuujcmWfmQp+oKIJZ+pIATkxVFvCQyPac=',2016-08,1.0,1,1,1,0.0
...,...,...,...,...,...,...,...
"49,995",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-03,115.0,115,115,115,0.0
"49,996",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-09,40.0,40,40,40,0.0
"49,997",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-10,149.0,149,149,149,0.0
"49,998",'KGrkTdr5HLYmGA/lMvOJSknlA2S6/9xip4SmRSo2iSE=',2016-12,86.0,86,86,86,0.0


In [33]:
return_agg_by_time(sample, 'msno', 'date_formatted', 'num_unq', resolution='M', every=6)

#,msno,date_formatted,avg_num_unq_6_month,sum_num_unq_6_month,max_num_unq_6_month,min_num_unq_6_month,std_num_unq_6_month
0,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',2014-12,21.666666666666668,65,28,12,6.944222218666545
1,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',2015-06,15.0,45,23,6,6.976149845485451
2,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',2015-12,31.0,93,47,16,12.675435561221033
3,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',2016-06,19.0,57,25,10,6.48074069840786
4,'QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=',2016-12,36.5,73,57,16,20.5
...,...,...,...,...,...,...,...
"26,494",'P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=',2014-12,23.333333333333332,70,61,1,26.787227470485924
"26,495",'P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=',2015-06,6.333333333333333,19,13,1,4.988876515698589
"26,496",'P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=',2015-12,18.666666666666668,56,28,3,11.145502331533656
"26,497",'P5atmaCDvQ6sBooufk6uLD/NVklEXJ4A/JWFiOkaz90=',2016-06,12.333333333333334,37,18,8,4.189935029992177


In [42]:
temp = return_agg_by_time(sample, 'msno', 'date_formatted', 'num_unq', resolution='M', every=3)

In [29]:
yearly_aggs_total_secs = return_agg_by_time(sample, 'msno', 'date_formatted', 'total_secs', resolution='Y', every=1)

In [35]:
t3 = yearly_aggs_total_secs.to_pandas_df()

In [41]:
t3.groupby('msno').tail(3)

,msno,date_formatted,avg_total_secs_1_year,sum_total_secs_1_year,max_total_secs_1_year,min_total_secs_1_year,std_total_secs_1_year
0,QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=,2015-01-01,5157.103000,36099.721,8817.947,1372.082,2385.354467
1,QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=,2016-01-01,8745.694500,52474.167,20013.525,2614.562,5915.981749
2,QiIddYgkK/kg64vUSceUgMkrfRdTD835iuRTx58KaYs=,2017-01-01,3755.116000,3755.116,3755.116,3755.116,0.000000
3,PjIozBWbtHFNte0hWowl7lkfoJyIHOkXffw2xKCXMzQ=,2015-01-01,1543.538000,4630.614,2414.502,756.971,679.310628
4,PjIozBWbtHFNte0hWowl7lkfoJyIHOkXffw2xKCXMzQ=,2016-01-01,1574.624500,6298.498,2876.080,1046.711,756.653734
...,...,...,...,...,...,...,...
17661,pxrhl5Ub4GcRP0w5OpPa9CnLZm7rsIcR1zN5xYNadkI=,2015-01-01,5281.913000,21127.652,16062.739,934.551,6241.471662
17662,pxrhl5Ub4GcRP0w5OpPa9CnLZm7rsIcR1zN5xYNadkI=,2016-01-01,8676.083000,17352.166,11375.424,5976.742,2699.341000
17663,kQbFgqHSLZeupySWMoBZpdfThEwkUZ6f9egJrKp5FuQ=,2015-01-01,26766.683667,160600.102,42556.736,8716.640,11425.052221
17664,kQbFgqHSLZeupySWMoBZpdfThEwkUZ6f9egJrKp5FuQ=,2016-01-01,16236.977200,81184.886,49621.556,5273.781,16927.209613


In [44]:
temp.groupby('msno').tail(3)

,msno,date_formatted,avg_num_unq_3_month,sum_num_unq_3_month,max_num_unq_3_month,min_num_unq_3_month,std_num_unq_3_month
6,RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=,2016-06-01,58.5,117,63,54,4.5
7,RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=,2016-09-01,82.5,165,102,63,19.5
8,RIJgWL2TuK6y58Ca3W/dfJGknBanXfe0fkMHllC/vJY=,2016-12-01,26.0,52,35,17,9.0
9,KsiXgnQvL6mb1DnuEF/eljW9cnyZrJTWr3Fc3VPQB9Y=,2014-12-01,2.0,2,2,2,0.0
10,KsiXgnQvL6mb1DnuEF/eljW9cnyZrJTWr3Fc3VPQB9Y=,2015-06-01,3.0,3,3,3,0.0
...,...,...,...,...,...,...,...
39112,RkQJydyfehW6xoiMl+56fzUpP2f6OdwkHDz165r554k=,2016-09-01,22.5,45,26,19,3.5
39113,RkQJydyfehW6xoiMl+56fzUpP2f6OdwkHDz165r554k=,2016-12-01,9.0,9,9,9,0.0
39120,ptJrO2x1bUg54fo7X51J/2kofVKuUga59xyiLwCJ5Io=,2016-06-01,59.0,59,59,59,0.0
39121,ptJrO2x1bUg54fo7X51J/2kofVKuUga59xyiLwCJ5Io=,2016-09-01,90.0,90,90,90,0.0
